In [4]:
import numpy as np
import model
from importlib import reload
reload(model)

<module 'model' from 'C:\\Users\\Paul\\Documents\\Keras-progressive_growing_of_gans\\Progressive growing of GANs\\model.py'>

In [2]:
G = model.Generator(use_wscale=False)

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [3]:
tw = model.replace_batch_norm(G)

Layer G1aBN_twin inserted after layer G1aBN
Layer G1bBN_twin inserted after layer G1bBN
Layer G2aBN_twin inserted after layer G2aBN
Layer G2bBN_twin inserted after layer G2bBN
Layer G3aBN_twin inserted after layer G3aBN
Layer G3bBN_twin inserted after layer G3bBN
Layer G4aBN_twin inserted after layer G4aBN
Layer G4bBN_twin inserted after layer G4bBN
